<a href="https://colab.research.google.com/github/HarmanBhutani/ML_projects/blob/main/Assignment4_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install contractions

     |████████████████████████████████| 266kB 34.8MB/s 
     |████████████████████████████████| 327kB 41.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85393 sha256=02b041b7738864fa34f8bba51f7ebae24d6405579365e50e064c5d61b267d37c
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [79]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import gensim
from gensim import corpora
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns

In [85]:
df_reut = pd.read_excel('reuters_news_headlines.xlsx', header=None)

In [86]:
df_reut.head()

,0,1,2
0,Headlines,Time,Description
1,TikTok considers London and other locations fo...,Jul 18 2020,TikTok has been in discussions with the UK gov...
2,Disney cuts ad spending on Facebook amid growi...,Jul 18 2020,Walt Disney has become the latest company to ...
3,Trail of missing Wirecard executive leads to B...,Jul 18 2020,Former Wirecard chief operating officer Jan M...
4,Twitter says attackers downloaded data from up...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...


In [87]:
df_reut[0]

0                                                Headlines
1        TikTok considers London and other locations fo...
2        Disney cuts ad spending on Facebook amid growi...
3        Trail of missing Wirecard executive leads to B...
4        Twitter says attackers downloaded data from up...
                               ...                        
32766    Malaysia says never hired British data firm at...
32767    Prosecutors search Volkswagen headquarters in ...
32768     McDonald's sets greenhouse gas reduction targets
32769    Pratt & Whitney to deliver spare A320neo engin...
32770    UK will always consider ways to improve data l...
Name: 0, Length: 32771, dtype: object

In [84]:
df_reut[0] = df_reut[0].apply(lambda x: ' '.join(x))

In [52]:
c = df_reut[0].apply(lambda x: ''.join(x))

In [88]:
nlp = en_core_web_sm.load()

In [ ]:
labels = []
for text in df_reut[0]:
    en = nlp(text).ents
    labels.append([l.label_ for l in en])

In [ ]:
labels[:15]

In [63]:
label_bool = []
for idx, lab in enumerate(labels):
    if len(lab)!=0:
        label_bool.append(True)
    else:
        label_bool.append(False)

In [64]:
df_reut['labels'] = labels

In [76]:
df_reut.head()

,0,1,2,labels
1,TikTok considers London and other locations fo...,Jul 18 2020,TikTok has been in discussions with the UK gov...,[GPE]
3,Trail of missing Wirecard executive leads to B...,Jul 18 2020,Former Wirecard chief operating officer Jan M...,"[ORG, GPE, GPE]"
4,Twitter says attackers downloaded data from up...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...,[CARDINAL]
5,U.S. Republicans seek liability protections as...,Jul 17 2020,A battle in the U.S. Congress over a new coron...,"[GPE, NORP]"
6,Wall Street Week Ahead: Fund managers navigate...,Jul 17 2020,Investors are searching for bargains in the wo...,"[EVENT, WORK_OF_ART]"


In [77]:
df_reut[0][9]

'Google bans ads on coronavirus conspiracy theory content'

In [78]:
df_reut = df_reut.loc[label_bool,:]

IndexError: ignored

In [68]:
mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(df_reut['labels']),
                   columns=mlb.classes_,
                   index=df_reut.index)

In [75]:
res.head(10)

,CARDINAL,PRODUCT,GPE,MONEY
1,0,0,1,0
3,0,0,1,0
4,1,0,0,0
5,0,0,1,0
6,0,0,0,0
7,1,0,0,0
8,0,0,1,0
9,0,0,0,0
10,0,0,1,0
11,0,0,0,0


In [74]:
res.sum()

CARDINAL     2656
PRODUCT      1087
GPE         14093
MONEY        2670
dtype: int64

In [73]:
res = res[['CARDINAL','ORG','DATE','MONEY']]

KeyError: ignored

In [ ]:
res = res[['ORG','DATE', 'CARDINAL','DATE']]

In [32]:
res.head()

,CARDINAL,PRODUCT,GPE,MONEY
8,0,0,1,0
10,1,0,0,0
12,1,0,0,0
18,0,1,0,0
19,0,1,0,0


In [34]:
from sklearn.model_selection import train_test_split

In [35]:
train, test = train_test_split(df_reut, test_size=0.3)

In [49]:
len(df_reut)

10117

In [50]:
len(train)

7081

In [51]:
len(test)

3036

In [39]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_features=150,norm='l2', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(train).toarray()

AttributeError: ignored

In [46]:
b = df_reut[0].apply(lambda x: ' '.join(x))

In [47]:
b

8        E       v       i       c       t       i     ...
10       F       l       i       g       h       t     ...
12       S       &       P               5       0     ...
18       E       x       c       l       u       s     ...
19       F       o       r       m       e       r     ...
                               ...                        
32752    U       K       '       s               C     ...
32759    U       K               i       n       v     ...
32763    K       e       n       y       a       '     ...
32765    E       U               d       e       m     ...
32770    U       K               w       i       l     ...
Name: 0, Length: 10117, dtype: object

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 200, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 51, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [53]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,max_features=150,norm='l2', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(c).toarray()

ValueError: ignored